In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import os

import time
import datetime
import calendar

In [3]:
def today_matchups():
    opts = Options()
    opts.add_argument("--headless")

    url = "https://www.cbssports.com/nba/injuries/"

    driver = webdriver.Chrome(options=opts)
    driver.set_page_load_timeout(20)
    time.sleep(1)
    driver.set_window_size(2100, 9000)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    date = datetime.datetime.today().strftime("%m-%d-%y")
    day = datetime.datetime.today().strftime("%A")

    games = soup.find_all("div", class_='Scoreboard__Row flex w-100 Scoreboard__Row__Main')
    games_data = []
    try:
        for game in games:
            game_data = {}
            away_tm = \
            game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[0]
            home_tm = \
            game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[1]
            game_data['Date'] = date
            game_data['Day'] = day
            
            try:
                game_data['Time'] = game.find("div", class_="ScoreCell__Time ScoreboardScoreCell__Time h9 clr-gray-03").text
            except:
                try:
                    game_data['Time'] = game.find("div", class_="ScoreCell__Time ScoreboardScoreCell__Time h9 clr-gray-01").text
                except: 
                    try:
                        game_data['Time'] = game.find("div", class_="ScoreCell__Time ScoreboardScoreCell__Time h9 clr-negative").text
                    except: pass
            
            try: 
                game_data['O/U'] = '{0:.1f}'.format(float(str(game.find("div", class_="Odds__Message").text).split(' ')[-1]))
                game_data['Line'] = str(game.find("div", class_="Odds__Message").text).lstrip('Line : ').split('O/U')[0]
            except: 
                game_data['O/U'] = 'In Prog.'
                game_data['Line'] = 'In Prog.'

            game_data["Away"] = away_tm.text
            # game_data["away_tm_abbr"] = team_names_nocity_dict[away_tm.text]

            try:
                game_data["Away Score"] = game.find_all("div", class_="ScoreCell__Score h4 clr-gray-01 fw-heavy tar ScoreCell_Score--scoreboard pl2")[0].text
            except:
                game_data['Away Score'] = 0

            game_data["Home"] = home_tm.text
            # game_data["home_tm_abbr"] = team_names_nocity_dict[home_tm.text]

            try:
                game_data["Home Score"] = game.find_all("div", class_="ScoreCell__Score h4 clr-gray-01 fw-heavy tar ScoreCell_Score--scoreboard pl2")[1].text
            except:
                game_data['Home Score'] = 0

            games_data.append(game_data)
    except: pass
    df = pd.DataFrame(games_data)
    driver.quit()

    return df

In [4]:
display(today_matchups())

""
